# Chapter 07: Building Trust and Transparency in LLMs

In an era where LLMs are becoming increasingly embedded in various facets of our daily lives, the need for trust and transparency has never been more critical. As these sophisticated systems take on roles ranging from customer service representatives to content creators, their influence grows, raising important questions about their reliability, decision-making processes, and ethical implications. We will begin by exploring how users can build trust in LLMs through transparency, allowing users to have confidence in the outputs and behaviors of LLMs.

In [1]:
import requests
from datetime import datetime, timedelta
from typing import Any
from pydantic import BaseModel, Field
from language_models.agent import (
    Agent,
    OutputType,
    PromptingStrategy,
    ReasoningStep,
)
from language_models.tools import Tool, current_date
from language_models.models.llm import OpenAILanguageModel
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [2]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=500,
    temperature=0.2,
)

## Governed Tool Access

Governed tool access entails implementing tiered access levels for tools, delineating permissions based on the tool’s nature and the associated risks. Tools may be classified into categories such as executable, requiring approval before execution, or merely suggestible. Achieving governed tool access involves embedding checks within the AI agent’s code to regulate tool usage. When an LLM opts to employ a particular tool, the software code verifies the tool’s permissions and determines the appropriate course of action based on the access level. For our purposes, we simply add a property to the tool object that specifies whether the tool requires approval or not.

In [4]:
class USGeopoliticalSurveyEarthquakeAPI(BaseModel):
    """Class that implements the API interface."""

    start_time: str = Field(
        None,
        description=(
            "Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format."
            + " Unless a timezone is specified, UTC is assumed."
        ),
    )
    end_time: str = Field(
        None,
        description=(
            "Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format."
            + " Unless a timezone is specified, UTC is assumed."
        ),
    )
    limit: int = Field(
        20000,
        description=(
            "Limit the results to the specified number of events. NOTE: The service limits queries to 20000,"
            + " and any that exceed this limit will generate a HTTP response code 400 Bad Request."
        ),
    )
    min_depth: int = Field(
        -100,
        description="Limit to events with depth more than the specified minimum.",
    )
    max_depth: int = Field(
        1000,
        description="Limit to events with depth less than the specified maximum.",
    )
    min_magnitude: int = Field(
        None,
        description="Limit to events with a magnitude larger than the specified minimum.",
    )
    max_magnitude: int = Field(
        None,
        description="Limit to events with a magnitude smaller than the specified maximum.",
    )
    alert_level: str = Field(
        None,
        description=(
            "Limit to events with a specific PAGER alert level."
            + " The allowed values are: alert_level=green Limit to events with PAGER"
            + ' alert level "green". alert_level=yellow Limit to events with PAGER alert level "yellow".'
            + ' alert_level=orange Limit to events with PAGER alert level "orange".'
            + ' alert_level=red Limit to events with PAGER alert level "red".'
        ),
    )

def get_earthquakes(
    endpoint: str,
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    params = {
        "format": "geojson",
        "starttime": start_time,
        "endtime": end_time,
        "limit": limit,
        "mindepth": min_depth,
        "maxdepth": max_depth,
        "minmagnitude": min_magnitude,
        "maxmagnitude": max_magnitude,
        "alertlevel": alert_level,
        "eventtype": "earthquake",
    }
    response = requests.get(
        f"https://earthquake.usgs.gov/fdsnws/event/1/{endpoint}",
        params=params,
        timeout=None,
    )
    return response.json()

def query_earthquakes(
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    return get_earthquakes(
        endpoint="query",
        start_time=start_time,
        end_time=end_time,
        limit=limit,
        min_depth=min_depth,
        max_depth=max_depth,
        min_magnitude=min_magnitude,
        max_magnitude=max_magnitude,
        alert_level=alert_level,
    )

def count_earthquakes(
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    return get_earthquakes(
        endpoint="count",
        start_time=start_time,
        end_time=end_time,
        limit=limit,
        min_depth=min_depth,
        max_depth=max_depth,
        min_magnitude=min_magnitude,
        max_magnitude=max_magnitude,
        alert_level=alert_level,
    )

query_earthquakes_tool = Tool(
    function=query_earthquakes,
    name="Query Earthquakes",
    description="Use this tool to search recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
    requires_approval=True,
)

count_earthquakes_tool = Tool(
    function=count_earthquakes,
    name="Count Earthquakes",
    description="Use this tool to count and aggregate recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
    requires_approval=True,
)

In [5]:
system_prompt = "You are an United States Geological Survey expert who can answer questions regarding earthquakes."

agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{question}",
    prompt_variables=["question"],
    output_type=OutputType.STRING,
    tools=[current_date, count_earthquakes_tool, query_earthquakes_tool],
    prompting_strategy=PromptingStrategy.CHAIN_OF_THOUGHT,
    verbose=True,
)

An example of approving the tool use: Count Earthquakes.

In [6]:
output = agent.invoke({"question": "How many earthquakes occurred today?"})

Thought: To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Count Earthquakes" tool for this. First, I need to get the current date.
Tool: Current Date
Tool Input: {}
Tool Output: 2024-08-12 12:51:21.099091
Thought: Now that I have the current date, I can use the "Count Earthquakes" tool to count the number of earthquakes that occurred today. The start time will be the start of the day (2024-08-12 00:00:00) and the end time will be the end of the day (2024-08-12 23:59:59).
Tool: Count Earthquakes
Tool Input: {'start_time': '2024-08-12T00:00:00', 'end_time': '2024-08-12T23:59:59'}
Tool Use Approved: Yes
Tool Output: {'count': 91, 'maxAllowed': 20000}
Thought: The output from the "Count Earthquakes" tool indicates that there were 91 earthquakes on the specified date.
Final Answer: There were 91 earthquakes today.


In [7]:
print(output.final_answer)

There were 91 earthquakes today.


An example of not approving the tool use: Query Earthquakes.

In [8]:
output = agent.invoke({"question": "Show me 3."})

Thought: To show the details of 3 earthquakes that occurred today, I can use the "Query Earthquakes" tool. I will limit the results to 3.
Tool: Query Earthquakes
Tool Input: {'start_time': '2024-08-12T00:00:00', 'end_time': '2024-08-12T23:59:59', 'limit': 3}
Tool Use Approved: No
Tool Output: The user did not approve the use of the tool: Query Earthquakes

Provide the final answer to the user's query
Thought: Since the user did not approve the use of the "Query Earthquakes" tool, I am unable to provide the details of the 3 earthquakes that occurred today.
Final Answer: I'm sorry, but I am unable to provide the details of the earthquakes without the use of the necessary tool.


In [9]:
print(output.final_answer)

I'm sorry, but I am unable to provide the details of the earthquakes without the use of the necessary tool.


## Displaying the Chain-of-Thought

In order to maintain transparency and provide users with insights into the AI agent’s decision-making process, it is essential to display the reasoning process of the LLM. This includes thoughts, tools used, and observations made during the interaction. By visualizing the thought-tool-observation loop, users can gain a better understanding of how the AI agent arrived at its final response. This transparency not only builds trust but also enables users to assess the reliability and effectiveness of the AI agent’s decision-making process. For instance, when viewing the Chain-of-Thought, users can examine whether the AI agent correctly provides tool inputs. If a user requests sales data for the past week, the input date range should ideally span from the start date of today minus 7 days to the end date of today, ensuring accurate data retrieval. Observing such details within the reasoning process empowers users to evaluate the AI agent’s adherence to input requirements and adjust the system prompt and task prompt accordingly. The same applies to workflows. On the other hand, when using Single Completion, agents generate a direct output in response to a user query, bypassing the reasoning process.

In [10]:
def render_reasoning(chain_of_thought: list[ReasoningStep]):
    names = {
        "prompt": "Prompt",
        "raw_output": "Raw Output",
        "observation": "Observation",
        "thought": "Thought",
        "final_answer": "Final Answer",
        "tool": "Tool",
    }
    for step in chain_of_thought:
        print(names[step.name])
        print("-" * len(step.name))
        if step.name in ("prompt", "raw_output", "thought", "observation", "final_answer"):
            print(step.content)
        else:
            print(f"Name: {step.content.name}")
            print(f"Input: {step.content.inputs}")
            print(f"Output: {step.content.output}")

        if step.name != "final_answer":
            print()

In earlier chapters, we observed the LLM's activities through the logs, focusing mainly on key aspects. However, in the Chain-of-Thought details, we gain access to more granular information, such as the prompt, raw output, and observations. Here is an example of the AI ​​agent using the Count Earthquakes tool, with the tool being approved.

In [11]:
output = agent.invoke({"question": "How many earthquakes occurred today?"})

Thought: To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Count Earthquakes" tool for this. First, I need to get the current date.
Tool: Current Date
Tool Input: {}
Tool Output: 2024-08-12 12:51:40.977439
Thought: Now that I have the current date, I can use the "Count Earthquakes" tool to count the number of earthquakes that occurred today. The start time will be the start of the day (2024-08-12 00:00:00) and the end time will be the end of the day (2024-08-12 23:59:59).
Tool: Count Earthquakes
Tool Input: {'start_time': '2024-08-12T00:00:00', 'end_time': '2024-08-12T23:59:59'}
Tool Use Approved: Yes
Tool Output: {'count': 91, 'maxAllowed': 20000}
Thought: The tool has already been used to determine the number of earthquakes that occurred today. The output was 91 earthquakes.
Final Answer: There were 91 earthquakes today.


In [12]:
print(output.final_answer)

There were 91 earthquakes today.


In [13]:
render_reasoning(output.chain_of_thought)

Prompt
------
How many earthquakes occurred today?

Raw Output
----------
Thought: To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Count Earthquakes" tool for this. First, I need to get the current date.

Tool: Current Date

Tool Input: {}

Thought
-------
To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Count Earthquakes" tool for this. First, I need to get the current date.

Tool
----
Name: Current Date
Input: {}
Output: 2024-08-12 12:51:40.977439

Raw Output
----------
Thought: Now that I have the current date, I can use the "Count Earthquakes" tool to count the number of earthquakes that occurred today. The start time will be the start of the day (2024-08-12 00:00:00) and the end time will be the end of the day (2024-08-12 23:59:59).

Tool: Count Earthquakes

Tool Input: {"start_time": "2024-08-12T00:00:00", "end_time": "2024-08-12T23:59:59"}

Thought
-------
Now that I ha

Here is an example of the AI ​​agent using the Query Earthquakes tool, with the tool being approved.

In [14]:
output = agent.invoke({"question": "Show me 3."})

Thought: To show the details of 3 earthquakes that occurred today, I can use the "Query Earthquakes" tool. I will limit the results to 3.
Tool: Query Earthquakes
Tool Input: {'start_time': '2024-08-12T00:00:00', 'end_time': '2024-08-12T23:59:59', 'limit': 3}
Tool Use Approved: No
Tool Output: The user did not approve the use of the tool: Query Earthquakes

Provide the final answer to the user's query
Thought: Since the user did not approve the use of the Query Earthquakes tool, I am unable to provide the details of the earthquakes.
Final Answer: I'm sorry, but I am unable to provide the details of the earthquakes without the use of the necessary tool.


In [15]:
print(output.final_answer)

I'm sorry, but I am unable to provide the details of the earthquakes without the use of the necessary tool.


In [16]:
render_reasoning(output.chain_of_thought)

Prompt
------
Show me 3.

Raw Output
----------
Thought: To show the details of 3 earthquakes that occurred today, I can use the "Query Earthquakes" tool. I will limit the results to 3.

Tool: Query Earthquakes

Tool Input: {"start_time": "2024-08-12T00:00:00", "end_time": "2024-08-12T23:59:59", "limit": 3}

Thought
-------
To show the details of 3 earthquakes that occurred today, I can use the "Query Earthquakes" tool. I will limit the results to 3.

Tool
----
Name: Query Earthquakes
Input: {'start_time': '2024-08-12T00:00:00', 'end_time': '2024-08-12T23:59:59', 'limit': 3}
Output: The user did not approve the use of the tool: Query Earthquakes

Provide the final answer to the user's query

Raw Output
----------
I'm sorry, but I am unable to provide the details of the earthquakes without the use of the necessary tool.

Observation
-----------
You made a mistake in your response:
I'm sorry, but I am unable to provide the details of the earthquakes without the use of the necessary tool.

## Monitoring

Monitoring plays a pivotal role in ensuring the effective operation and reliability of LLMs. Given the intricate and evolving nature of these systems, continuous oversight is essential to detect anomalies, resolve performance issues, and maintain user trust. One fundamental aspect of monitoring involves logging every interaction between users and the AI agent. This comprehensive logging provides transparency, allowing users to review past decisions and understand the context in which they were made and by whom. It also facilitates accountability, allowing developers to identify and address misuse or violations of the system.

Additionally, monitoring encompasses the evaluation of tool utilization within the AI agent. By tracking which tools are frequently or infrequently used, developers can make informed decisions about system optimization. Tools that are rarely utilized may be removed to streamline the system and reduce costs, while those frequently used can be prioritized for enhancements to improve their effectiveness. Visualizing tool usage through graphical representations provides further insights into the AI agent's behavior, helping to identify whether the system is functioning as expected or if performance issues need to be addressed. This approach allows developers to fine-tune the AI’s operations based on actual usage patterns and user interactions.

Moreover, monitoring also involves assessing the economic aspects of operating LLMs. By tracking the consumption of tokens and other resources, users can evaluate the cost-effectiveness of their AI agents. This financial oversight helps in making strategic decisions about transitioning to more cost-efficient models while balancing performance needs. Overall, a comprehensive monitoring strategy, including tool usage analysis, financial tracking, and transparent logging, is essential for maintaining trust and transparency in LLMs. These practices not only uphold ethical standards but also enable developers to continuously improve and adapt their AI solutions to meet user expectations and operational requirements.